<a href="https://colab.research.google.com/github/micaelCZ/Paper_Repositorio/blob/main/PAPER_Escenario_TORNOTOR_B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Escenario TORNOTOR B





In [14]:
#Librerias
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import cross_validate

In [15]:
# Cargar los datos
url = 'https://raw.githubusercontent.com/micaelCZ/Paper_Repositorio/main/dataset/datasetPreprocesado/Escenario2.csv'
df = pd.read_csv(url, low_memory=False)
y = df['label']
label_map = {'AUDIO': 0, 'BROWSING': 1, 'CHAT': 2, 'FILE-TRANSFER': 3, 'MAIL': 4, 'P2P': 5, 'VIDEO': 6, 'VOIP': 7}
change_labels = lambda x: label_map[x] if x in label_map else -1
y = df['label'].apply(change_labels)
X = df.iloc[:, 4:-1]

In [16]:
# Normalizar los datos
def dfNormalize(df):
    for feature_name in df.columns:
        df.loc[:, feature_name] = pd.to_numeric(df.loc[:, feature_name], errors='coerce').fillna(0)
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()   
        if (max_value - min_value) > 0:
            df.loc[:, feature_name] = (df.loc[:, feature_name] - min_value) / (max_value - min_value)
        else:
            df.loc[:, feature_name] = (df.loc[:, feature_name] - min_value)    
    return df

X = dfNormalize(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

imputer = SimpleImputer(strategy='median')
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_test_scaled = scaler.transform(X_test_imputed)

In [9]:
# Entrenar un modelo SVM utilizando SVC de sklearn con One-vs-One (OvO)
ovo_svm_linear = SVC(kernel='linear', C=0.1, random_state=42, decision_function_shape='ovo')
ovo_scores_linear = cross_validate(ovo_svm_linear, X_train_scaled, y_train, cv=5, scoring=['accuracy', 'precision_micro', 'recall_micro', 'f1_micro'])

ovo_svm_poly = SVC(kernel='poly', degree=3, C=0.1, random_state=42, decision_function_shape='ovo')
ovo_scores_poly = cross_validate(ovo_svm_poly, X_train_scaled, y_train, cv=5, scoring=['accuracy', 'precision_micro', 'recall_micro', 'f1_micro'])

ovo_svm_rbf = SVC(kernel='rbf', C=0.1, random_state=42, decision_function_shape='ovo')
ovo_scores_rbf = cross_validate(ovo_svm_rbf, X_train_scaled, y_train, cv=5, scoring=['accuracy', 'precision_micro', 'recall_micro', 'f1_micro'])

ovo_svm_sigmoid = SVC(kernel='sigmoid', C=0.1, random_state=42, decision_function_shape='ovo')
ovo_scores_sigmoid = cross_validate(ovo_svm_sigmoid, X_train_scaled, y_train, cv=5, scoring=['accuracy', 'precision_micro', 'recall_micro', 'f1_micro'])

In [10]:
# Imprimir resultados de validación cruzada con OvO - Kernel Linear
print('Resultados de validación cruzada con One-vs-One (OvO) - kernel linear:')
print('Accuracy: {:.2f}%'.format(ovo_scores_linear['test_accuracy'].mean() * 100))
print('Precision: {:.2f}%'.format(ovo_scores_linear['test_precision_micro'].mean() * 100))
print('Recall: {:.2f}%'.format(ovo_scores_linear['test_recall_micro'].mean() * 100))
print('F1-score: {:.2f}%'.format(ovo_scores_linear['test_f1_micro'].mean() * 100))

Resultados de validación cruzada con One-vs-One (OvO) - kernel linear:
Accuracy: 59.07%
Precision: 59.07%
Recall: 59.07%
F1-score: 59.07%


In [11]:
# Imprimir resultados de validación cruzada con OvO - Kernel Poly
print('Resultados de validación cruzada con One-vs-One (OvO) - kernel poly:')
print('Accuracy: {:.2f}%'.format(ovo_scores_poly['test_accuracy'].mean() * 100))
print('Precision: {:.2f}%'.format(ovo_scores_poly['test_precision_micro'].mean() * 100))
print('Recall: {:.2f}%'.format(ovo_scores_poly['test_recall_micro'].mean() * 100))
print('F1-score: {:.2f}%'.format(ovo_scores_poly['test_f1_micro'].mean() * 100))

Resultados de validación cruzada con One-vs-One (OvO) - kernel poly:
Accuracy: 34.44%
Precision: 34.44%
Recall: 34.44%
F1-score: 34.44%


In [12]:
# Imprimir resultados de validación cruzada con OvO - Kernel Rbf
print('Resultados de validación cruzada con One-vs-One (OvO) - kernel rbf:')
print('Accuracy: {:.2f}%'.format(ovo_scores_rbf['test_accuracy'].mean() * 100))
print('Precision: {:.2f}%'.format(ovo_scores_rbf['test_precision_micro'].mean() * 100))
print('Recall: {:.2f}%'.format(ovo_scores_rbf['test_recall_micro'].mean() * 100))
print('F1-score: {:.2f}%'.format(ovo_scores_rbf['test_f1_micro'].mean() * 100))

Resultados de validación cruzada con One-vs-One (OvO) - kernel rbf:
Accuracy: 46.88%
Precision: 46.88%
Recall: 46.88%
F1-score: 46.88%


In [13]:
# Imprimir resultados de validación cruzada con OvO - Kernel Sigmoid
print('Resultados de validación cruzada con One-vs-One (OvO) - kernel sigmoid:')
print('Accuracy: {:.2f}%'.format(ovo_scores_sigmoid['test_accuracy'].mean() * 100))
print('Precision: {:.2f}%'.format(ovo_scores_sigmoid['test_precision_micro'].mean() * 100))
print('Recall: {:.2f}%'.format(ovo_scores_sigmoid['test_recall_micro'].mean() * 100))
print('F1-score: {:.2f}%'.format(ovo_scores_sigmoid['test_f1_micro'].mean() * 100))

Resultados de validación cruzada con One-vs-One (OvO) - kernel sigmoid:
Accuracy: 42.49%
Precision: 42.49%
Recall: 42.49%
F1-score: 42.49%
